Data sources from https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/
Excercise in producing a decision tree to predict forest cover of parcels of land based on the 50+ tributes.
Based on the case study in Advnaced Analytics using Spark
Using Almond Shell kernel for Jupyter notebook

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3`
import org.apache.spark.sql._

import $ivy.$                                  

import org.apache.spark.sql._

In [2]:
import org.apache.log4j.{Logger,Level}

import org.apache.log4j.{Logger,Level}

In [3]:
Logger.getLogger("org").setLevel(Level.ERROR)

Create the sql sessions 

In [4]:
val spark = {
            org.apache.spark.sql.NotebookSparkSession.builder()
            .master("local[*]")
            .getOrCreate()
            }

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@11473bbc

In [5]:
val data = spark.read
            .option("header","false")
            .option("inferSchema","true")
            .csv("../covtype/")

csv at cmd4.sc:4

1 / 1

csv at cmd4.sc:4

8 / 8

data: DataFrame = [_c0: string, _c1: string ... 53 more fields]

In [6]:
data.count()

count at cmd5.sc:1

8 / 8

count at cmd5.sc:1

1 / 1

res5: Long = 581259L

In [7]:
data.head()

head at cmd6.sc:1

1 / 1

res6: Row = [2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5]

In [8]:
data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: integer (nullable = true)
 |-- _c15: integer (nullable = true)
 |-- _c16: integer (nullable = true)
 |-- _c17: integer (nullable = true)
 |-- _c18: integer (nullable = true)
 |-- _c19: integer (nullable = true)
 |-- _c20: integer (nullable = true)
 |-- _c21: integer (nullable = true)
 |-- _c22: integer (nullable = true)
 |-- _c23: integer (nullable = true)
 |-- _c24: integer (nullable = true)
 |-- _c25: integer (nullable = true)
 |-- _c26: integer (nullable = true)
 |-- _c27

Looks like the schema isn't what was expected. _c0..2 should be integers

In [9]:
import org.apache.spark.sql.types._

import org.apache.spark.sql.types._

Have to update the scheme. Only way seems to be to create new DF every time.

In [10]:
val data2=data.withColumn("_c0",data("_c0").cast(IntegerType))
val data3=data2.withColumn("_c1",data2("_c1").cast(IntegerType))
val data4=data3.withColumn("_c2",data3("_c2").cast(IntegerType))

data2: DataFrame = [_c0: int, _c1: string ... 53 more fields]
data3: DataFrame = [_c0: int, _c1: int ... 53 more fields]
data4: DataFrame = [_c0: int, _c1: int ... 53 more fields]

In [11]:
data4.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: integer (nullable = true)
 |-- _c15: integer (nullable = true)
 |-- _c16: integer (nullable = true)
 |-- _c17: integer (nullable = true)
 |-- _c18: integer (nullable = true)
 |-- _c19: integer (nullable = true)
 |-- _c20: integer (nullable = true)
 |-- _c21: integer (nullable = true)
 |-- _c22: integer (nullable = true)
 |-- _c23: integer (nullable = true)
 |-- _c24: integer (nullable = true)
 |-- _c25: integer (nullable = true)
 |-- _c26: integer (nullable = true)
 |-- _

Above approach worked by duplicating the dataframe everytime seem thereis potential for memory overflow. Try doing it when adding titles.

Create header titles for the columns. 

In [12]:
val colNames = Seq(
"Elevation", "Aspect", "Slope",
"Horizontal_Distance_To_Hydrology", "Vertical_Distance_To_Hydrology",
"Horizontal_Distance_To_Roadways",
"Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm",
"Horizontal_Distance_To_Fire_Points"
) ++ (
(0 until 4).map(i => s"Wilderness_Area_$i")
) ++ (
(0 until 40).map(i => s"Soil_Type_$i")
) ++ Seq("Cover_Type")

colNames: Seq[String] = List(
  "Elevation",
  "Aspect",
  "Slope",
  "Horizontal_Distance_To_Hydrology",
  "Vertical_Distance_To_Hydrology",
  "Horizontal_Distance_To_Roadways",
  "Hillshade_9am",
  "Hillshade_Noon",
  "Hillshade_3pm",
  "Horizontal_Distance_To_Fire_Points",
  "Wilderness_Area_0",
  "Wilderness_Area_1",
  "Wilderness_Area_2",
  "Wilderness_Area_3",
  "Soil_Type_0",
  "Soil_Type_1",
  "Soil_Type_2",
  "Soil_Type_3",
  "Soil_Type_4",
  "Soil_Type_5",
  "Soil_Type_6",
  "Soil_Type_7",
  "Soil_Type_8",
  "Soil_Type_9",
  "Soil_Type_10",
  "Soil_Type_11",
  "Soil_Type_12",
  "Soil_Type_13",
  "Soil_Type_14",
  "Soil_Type_15",
  "Soil_Type_16",
  "Soil_Type_17",
  "Soil_Type_18",
  "Soil_Type_19",
  "Soil_Type_20",
  "Soil_Type_21",
  "Soil_Type_22",
  "Soil_Type_23",
...

In [13]:
colNames.getClass

res12: Class[T] = class scala.collection.immutable.$colon$colon

implicits contains the function $ which returns a column object. Needed to access the data type in order to cast it to integer. $("string"): column   ... pretty 'hacky' syntax.

In [14]:
import spark.implicits._

import spark.implicits._

In [15]:
val dataClean = data.toDF(colNames:_*).
    withColumn("Elevation",$"Elevation".cast(IntegerType))
    .withColumn("Aspect",$"Aspect".cast(IntegerType))
    .withColumn("Slope",$"Slope".cast(IntegerType))
    .withColumn("Cover_Type",$"Cover_Type".cast(DoubleType))


dataClean: DataFrame = [Elevation: int, Aspect: int ... 53 more fields]

In [16]:
dataClean.show() // Using the spark.sql API

show at cmd15.sc:1

1 / 1

+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+----------------------------------+-----------------+-----------------+-----------------+-----------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+----------+
|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Hillshade_9am|Hillshade_Noon|Hillshade_3pm|Horizontal_Distance_To_Fire_Points|Wilderness

|     2617|    45|    9|                             240|                            56|                            666|          223|           221|          133|                              6244|                1|                0|                0|                0|          0|          0|          0|          0|          0|          0|          0|          0|          0|          0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           1|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|       5.0|
|     2612|    59|   10|                             247|                            11|                            636|          228|           219|          124|                              6230|          

|     2504|     0|    4|                              95|                             5|                            691|          214|           232|          156|                              5572|                1|                0|                0|                0|          0|          0|          0|          0|          0|          0|          0|          0|          0|          0|           0|           0|           0|           0|           0|           0|           0|           1|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|       5.0|
|     2503|    38|    5|                              85|                            10|                            741|          220|           228|          144|                              5555|          

Randomly split the data into train and test using 10% for testing (will need to further split the training data if CV performed

In [17]:
val Array(trainData, testData) = dataClean.randomSplit(Array(0.9,0.1))
trainData.cache()
testData.cache()

trainData: Dataset[Row] = [Elevation: int, Aspect: int ... 53 more fields]
testData: Dataset[Row] = [Elevation: int, Aspect: int ... 53 more fields]
res16_1: Dataset[Row] = [Elevation: int, Aspect: int ... 53 more fields]
res16_2: Dataset[Row] = [Elevation: int, Aspect: int ... 53 more fields]

Vectorise the features. This is required by all SparkML alogorithms.

Took some messing around to find the right repo for ml lib. Eventually found a reference on the Mvnrepository site to find the mllib version that goes with scala 2.12.

In [18]:
import $ivy.`org.apache.spark::spark-mllib:2.4.0`

import $ivy.$                                    

In [19]:
import org.apache.spark.ml.feature.VectorAssembler

import org.apache.spark.ml.feature.VectorAssembler

In [20]:
val inputCols = trainData.columns.filter(_!="Cover_Type")
val assembler = new org.apache.spark.ml.feature.VectorAssembler()
                    .setInputCols(inputCols)
                    .setOutputCol("featureVector")

inputCols: Array[String] = Array(
  "Elevation",
  "Aspect",
  "Slope",
  "Horizontal_Distance_To_Hydrology",
  "Vertical_Distance_To_Hydrology",
  "Horizontal_Distance_To_Roadways",
  "Hillshade_9am",
  "Hillshade_Noon",
  "Hillshade_3pm",
  "Horizontal_Distance_To_Fire_Points",
  "Wilderness_Area_0",
  "Wilderness_Area_1",
  "Wilderness_Area_2",
  "Wilderness_Area_3",
  "Soil_Type_0",
  "Soil_Type_1",
  "Soil_Type_2",
  "Soil_Type_3",
  "Soil_Type_4",
  "Soil_Type_5",
  "Soil_Type_6",
  "Soil_Type_7",
  "Soil_Type_8",
  "Soil_Type_9",
  "Soil_Type_10",
  "Soil_Type_11",
  "Soil_Type_12",
  "Soil_Type_13",
  "Soil_Type_14",
  "Soil_Type_15",
  "Soil_Type_16",
  "Soil_Type_17",
  "Soil_Type_18",
  "Soil_Type_19",
  "Soil_Type_20",
  "Soil_Type_21",
  "Soil_Type_22",
  "Soil_Type_23",
...
assembler: VectorAssembler = vecAssembler_5e9bf1988912

Since spark 2.4 vector assembler extends HadHandleInvalid which requires you to specify how nulls will be handled. It take a single parameter ("skip") to skip when encountered. If not used then the dataset must be cleaned up and the missing data imputed. 

See stackoverflow
https://stackoverflow.com/questions/41362295/sparkexception-values-to-assemble-cannot-be-null


TODO: Investigate the source of nulls

In [21]:
val assembledTrainData = assembler.setHandleInvalid("skip").transform(trainData)

assembledTrainData: DataFrame = [Elevation: int, Aspect: int ... 54 more fields]

In [22]:
assembledTrainData.select("featureVector").show(truncate=false)

show at cmd21.sc:1

1 / 1

+-----------------------------------------------------------------------------------------------------+
|featureVector                                                                                        |
+-----------------------------------------------------------------------------------------------------+
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1863.0,37.0,17.0,120.0,18.0,90.0,217.0,202.0,115.0,769.0,1.0,1.0])  |
|(54,[0,1,2,5,6,7,8,9,13,18],[1874.0,18.0,14.0,90.0,208.0,209.0,135.0,793.0,1.0,1.0])                 |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1879.0,28.0,19.0,30.0,12.0,95.0,209.0,196.0,117.0,778.0,1.0,1.0])   |
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1888.0,33.0,22.0,150.0,46.0,108.0,209.0,185.0,103.0,735.0,1.0,1.0]) |
|(54,[0,1,2,3,4,5,6,7,8,9,13,14],[1889.0,28.0,22.0,150.0,23.0,120.0,205.0,185.0,108.0,759.0,1.0,1.0]) |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1889.0,353.0,30.0,95.0,39.0,67.0,153.0,172.0,146.0,600.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1896.0,337.0,12.0,30.0,6.0,175

In [23]:
assembledTrainData.getClass

res22: Class[T] = class org.apache.spark.sql.Dataset

In [24]:
trainData.getClass

res23: Class[T] = class org.apache.spark.sql.Dataset

Note on above. The transform fucntion converts a dataframe to a dataframe but re-organises it.

Now to create the DecisionTreem classifier for predicting the group cover type (a categorical valiable ranging 1..7)

In [25]:
import org.apache.spark.ml.classification.DecisionTreeClassifier
import scala.util.Random

import org.apache.spark.ml.classification.DecisionTreeClassifier

import scala.util.Random

In [26]:
val classifier = new org.apache.spark.ml.classification.DecisionTreeClassifier()
                        .setSeed(Random.nextLong())
                        .setLabelCol("Cover_Type")
                        .setFeaturesCol("featureVector")
                        .setPredictionCol("prediction")


classifier: DecisionTreeClassifier = dtc_47153b85c161

In [27]:
val model = classifier.fit(assembledTrainData)
//print(model.toDebugString)

take at Classifier.scala:111

8 / 8

take at Classifier.scala:111

1 / 1

take at DecisionTreeMetadata.scala:112

...

mapPartitions at RandomForest.scala:538

8 / 8

collectAsMap at RandomForest.scala:557

8 / 8

model: org.apache.spark.ml.classification.DecisionTreeClassificationModel = DecisionTreeClassificationModel (uid=dtc_47153b85c161) of depth 5 with 41 nodes

In [28]:
print(model.toDebugString)

DecisionTreeClassificationModel (uid=dtc_47153b85c161) of depth 5 with 41 nodes
  If (feature 0 <= 3050.5)
   If (feature 0 <= 2555.5)
    If (feature 10 <= 0.5)
     If (feature 0 <= 2450.5)
      If (feature 3 <= 15.0)
       Predict: 4.0
      Else (feature 3 > 15.0)
       Predict: 3.0
     Else (feature 0 > 2450.5)
      If (feature 17 <= 0.5)
       Predict: 2.0
      Else (feature 17 > 0.5)
       Predict: 3.0
    Else (feature 10 > 0.5)
     Predict: 2.0
   Else (feature 0 > 2555.5)
    If (feature 0 <= 2954.5)
     If (feature 15 <= 0.5)
      If (feature 17 <= 0.5)
       Predict: 2.0
      Else (feature 17 > 0.5)
       Predict: 3.0
     Else (feature 15 > 0.5)
      Predict: 3.0
    Else (feature 0 > 2954.5)
     If (feature 3 <= 191.0)
      If (feature 36 <= 0.5)
       Predict: 2.0
      Else (feature 36 > 0.5)
       Predict: 1.0
     Else (feature 3 > 191.0)
      Predict: 2.0
  Else (feature 0 > 3050.5)
   If (feature 0 <= 3317.5)
    If (feature 7 <= 240.5)
     Pred

Cool! Above cell appears to show a recursive definition (and almost self explaining) of the decision tree model.
So now the model has been fit (estimator(ed) in spark speak) we can use the model to make predictions on the test data. 
Note that the default settings have been used on the above model so we don't expect this out of the box model to do very well.

Without getting under the hood I assume that the decison tree optimises the gini index for the tree. TODO find out.

DecisiontTreeClassifiers have an 'importance' dunction to assess the importance of each feature. Not obvious what this is but assume its some sort of sensitivity analysis.??

In [29]:
model.featureImportances.toArray.zip(inputCols).sorted.reverse.foreach(println)

(0.8076689665257774,Elevation)
(0.04080031878885087,Horizontal_Distance_To_Hydrology)
(0.031839310791053246,Soil_Type_3)
(0.028596409181629895,Wilderness_Area_0)
(0.025515390871204253,Soil_Type_1)
(0.025428863651643756,Hillshade_Noon)
(0.017794972623845235,Soil_Type_31)
(0.01177942106451395,Wilderness_Area_2)
(0.005881419786712523,Soil_Type_22)
(0.002522603561809275,Hillshade_3pm)
(0.002172323152959786,Horizontal_Distance_To_Roadways)
(0.0,Wilderness_Area_3)
(0.0,Wilderness_Area_1)
(0.0,Vertical_Distance_To_Hydrology)
(0.0,Soil_Type_9)
(0.0,Soil_Type_8)
(0.0,Soil_Type_7)
(0.0,Soil_Type_6)
(0.0,Soil_Type_5)
(0.0,Soil_Type_4)
(0.0,Soil_Type_39)
(0.0,Soil_Type_38)
(0.0,Soil_Type_37)
(0.0,Soil_Type_36)
(0.0,Soil_Type_35)
(0.0,Soil_Type_34)
(0.0,Soil_Type_33)
(0.0,Soil_Type_32)
(0.0,Soil_Type_30)
(0.0,Soil_Type_29)
(0.0,Soil_Type_28)
(0.0,Soil_Type_27)
(0.0,Soil_Type_26)
(0.0,Soil_Type_25)
(0.0,Soil_Type_24)
(0.0,Soil_Type_23)
(0.0,Soil_Type_21)
(0.0,Soil_Type_20)
(0.0,Soil_Type_2)
(0.0,Soi

So the above model has the elevation as by far and away the most important feature. However we have not assessed the performance of modey yet so this should be treated cautiously until this is done. 

Note that the decisionTreeClassificationModel is a transformed ( in spark speak this is just a function that transform a dataframe into a dataframe

In [30]:
val preds = model.transform(assembledTrainData)

preds.select("Cover_Type","prediction","probability").show(truncate=false)

show at cmd29.sc:3

1 / 1

+----------+----------+-------------------------------------------------------------------------------------------------+
|Cover_Type|prediction|probability                                                                                      |
+----------+----------+-------------------------------------------------------------------------------------------------+
|6.0       |3.0       |[0.0,0.0,0.033148026656538106,0.6327820171955388,0.05072338662339008,0.0,0.28334656952453297,0.0]|
|6.0       |4.0       |[0.0,0.0,0.04150620453572957,0.29097133076593923,0.43046640992725715,0.0,0.23705605477107403,0.0]|
|6.0       |3.0       |[0.0,0.0,0.033148026656538106,0.6327820171955388,0.05072338662339008,0.0,0.28334656952453297,0.0]|
|6.0       |3.0       |[0.0,0.0,0.033148026656538106,0.6327820171955388,0.05072338662339008,0.0,0.28334656952453297,0.0]|
|6.0       |3.0       |[0.0,0.0,0.033148026656538106,0.6327820171955388,0.05072338662339008,0.0,0.28334656952453297,0.0]|
|6.0       |3.0       |[

preds: DataFrame = [Elevation: int, Aspect: int ... 57 more fields]

In [31]:
preds.columns.foreach(println)

Elevation
Aspect
Slope
Horizontal_Distance_To_Hydrology
Vertical_Distance_To_Hydrology
Horizontal_Distance_To_Roadways
Hillshade_9am
Hillshade_Noon
Hillshade_3pm
Horizontal_Distance_To_Fire_Points
Wilderness_Area_0
Wilderness_Area_1
Wilderness_Area_2
Wilderness_Area_3
Soil_Type_0
Soil_Type_1
Soil_Type_2
Soil_Type_3
Soil_Type_4
Soil_Type_5
Soil_Type_6
Soil_Type_7
Soil_Type_8
Soil_Type_9
Soil_Type_10
Soil_Type_11
Soil_Type_12
Soil_Type_13
Soil_Type_14
Soil_Type_15
Soil_Type_16
Soil_Type_17
Soil_Type_18
Soil_Type_19
Soil_Type_20
Soil_Type_21
Soil_Type_22
Soil_Type_23
Soil_Type_24
Soil_Type_25
Soil_Type_26
Soil_Type_27
Soil_Type_28
Soil_Type_29
Soil_Type_30
Soil_Type_31
Soil_Type_32
Soil_Type_33
Soil_Type_34
Soil_Type_35
Soil_Type_36
Soil_Type_37
Soil_Type_38
Soil_Type_39
Cover_Type
featureVector
rawPrediction
probability
prediction


In [32]:
val results = preds.select("Cover_Type","prediction")

results: DataFrame = [Cover_Type: double, prediction: double]

In [33]:
results.selectExpr("Cover_Type==prediction as Matched","Cover_Type")
    .groupBy("Cover_Type").pivot("Matched",Seq("true","false")).count().show()

show at cmd32.sc:2

8 / 8

show at cmd32.sc:2

200 / 200

show at cmd32.sc:2

...

show at cmd32.sc:2

100 / 100

show at cmd32.sc:2

75 / 75

+----------+------+-----+
|Cover_Type|  true|false|
+----------+------+-----+
|       7.0| 10381| 8134|
|       1.0|122935|67830|
|       4.0|  1006| 1488|
|       3.0| 25912| 6438|
|       2.0|205774|48967|
|       6.0|  null|15615|
|       5.0|  null| 8536|
+----------+------+-----+



In [34]:
val predPivot = results.selectExpr("Cover_Type","prediction")
    .groupBy("Cover_Type")
    .pivot("prediction",(1 to 7))
    .count()
    .na.fill(0).sort("Cover_Type")

predPivot: Dataset[Row] = [Cover_Type: double, 1: bigint ... 6 more fields]

In [35]:
predPivot.show()

show at cmd34.sc:1

8 / 8

show at cmd34.sc:1

200 / 200

show at cmd34.sc:1

200 / 200

+----------+------+------+-----+----+---+---+-----+
|Cover_Type|     1|     2|    3|   4|  5|  6|    7|
+----------+------+------+-----+----+---+---+-----+
|       1.0|122935| 62339|  171|   0|  0|  0| 5320|
|       2.0| 43866|205774| 4312|  97|  0|  0|  692|
|       3.0|     0|  5758|25912| 680|  0|  0|    0|
|       4.0|     0|    19| 1469|1006|  0|  0|    0|
|       5.0|     7|  7793|  736|   0|  0|  0|    0|
|       6.0|     0|  6141| 8920| 554|  0|  0|    0|
|       7.0|  7798|   277|   59|   0|  0|  0|10381|
+----------+------+------+-----+----+---+---+-----+



TODO: Add row total

In [36]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

In [37]:
val evaluator = new org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator()
                .setLabelCol("Cover_Type")
                .setPredictionCol("prediction")

evaluator: MulticlassClassificationEvaluator = mcEval_bc990ae0e8e3

In [38]:
evaluator.setMetricName("accuracy").evaluate(preds)

map at MulticlassMetrics.scala:45

8 / 8

collectAsMap at MulticlassMetrics.scala:47

8 / 8

countByValue at MulticlassMetrics.scala:42

8 / 8

countByValue at MulticlassMetrics.scala:42

8 / 8

res37: Double = 0.6998026829007143

As expected accuracy not great. Also, looking at the prediction pivot table shows that there is class imbalance that might be affecting the utility of accuracy and f1 might be a better score.


In [39]:
evaluator.setMetricName("f1").evaluate(preds)

countByValue at MulticlassMetrics.scala:42

8 / 8

countByValue at MulticlassMetrics.scala:42

8 / 8

map at MulticlassMetrics.scala:45

...

map at MulticlassMetrics.scala:50

8 / 8

collectAsMap at MulticlassMetrics.scala:52

8 / 8

res38: Double = 0.6811039017892407

Actually not that difference suggesting that the algorithm (with the current set of parameters) isn't greatly affected by the class imbalance. This should be monitored on going as the imbalance of the data warrants this.

Its seems that spark has a confusion matrix ( what I produced in the above pivot table).Its accesses (in the book at least) via the Mllib rdd API (which is in maintenance mode now with the dataframe spark.ml api being the promoted one)

In [40]:
import org.apache.spark.mllib.evaluation.MulticlassMetrics
val predictionRDD = preds.
        select("prediction","Cover_Type").
        as[(Double,Double)].
        rdd
val multiclassMetrics = new org.apache.spark.mllib.evaluation.MulticlassMetrics(predictionRDD)

import org.apache.spark.mllib.evaluation.MulticlassMetrics

predictionRDD: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[132] at rdd at cmd39.sc:4
multiclassMetrics: MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@19058212

In [41]:
multiclassMetrics.confusionMatrix

map at MulticlassMetrics.scala:45

8 / 8

collectAsMap at MulticlassMetrics.scala:47

8 / 8

map at MulticlassMetrics.scala:55

8 / 8

collectAsMap at MulticlassMetrics.scala:57

8 / 8

res40: org.apache.spark.mllib.linalg.Matrix = 122935.0  62339.0   171.0    0.0     0.0  0.0  5320.0   
43866.0   205774.0  4312.0   97.0    0.0  0.0  692.0    
0.0       5758.0    25912.0  680.0   0.0  0.0  0.0      
0.0       19.0      1469.0   1006.0  0.0  0.0  0.0      
7.0       7793.0    736.0    0.0     0.0  0.0  0.0      
0.0       6141.0    8920.0   554.0   0.0  0.0  0.0      
7798.0    277.0     59.0     0.0     0.0  0.0  10381.0  

In [42]:
predPivot.show()

show at cmd41.sc:1

8 / 8

show at cmd41.sc:1

200 / 200

show at cmd41.sc:1

200 / 200

+----------+------+------+-----+----+---+---+-----+
|Cover_Type|     1|     2|    3|   4|  5|  6|    7|
+----------+------+------+-----+----+---+---+-----+
|       1.0|122935| 62339|  171|   0|  0|  0| 5320|
|       2.0| 43866|205774| 4312|  97|  0|  0|  692|
|       3.0|     0|  5758|25912| 680|  0|  0|    0|
|       4.0|     0|    19| 1469|1006|  0|  0|    0|
|       5.0|     7|  7793|  736|   0|  0|  0|    0|
|       6.0|     0|  6141| 8920| 554|  0|  0|    0|
|       7.0|  7798|   277|   59|   0|  0|  0|10381|
+----------+------+------+-----+----+---+---+-----+



To evaluate the 70% accuracy from the default model we can benchmark it against a null model and use an inner product of the class probabilities to compare.

In [43]:
def classProbs(df: DataFrame):Array[Double] = {
    val totalCounts = df.count().asInstanceOf[Double]
        df.groupBy("Cover_Type").count()
        .orderBy("Cover_Type")
        .select("count").as[Double]
        .map(_/totalCounts)
        .collect()
}

defined function classProbs

Take inner product of test and train class probabilities as A measure of similarity

In [44]:
val trainProbs = classProbs(trainData)
val testProbs = classProbs(testData)
trainProbs.zip(testProbs).map(x=>x._1*x._2).sum

count at cmd42.sc:2

8 / 8

count at cmd42.sc:2

1 / 1

collect at cmd42.sc:6

...

collect at cmd42.sc:6

200 / 200

collect at cmd42.sc:6

8 / 8

trainProbs: Array[Double] = Array(
  4.147291933039392E-4,
  0.3645890071918247,
  0.4868595826333584,
  0.06182714010775314,
  0.004766518931336517,
  0.0163139557329144,
  0.029843301167930922,
  0.03538576504157803
)
testProbs: Array[Double] = Array(
  5.170096163788646E-4,
  0.3631992555061524,
  0.49219315479267917,
  0.058663357805121844,
  0.004360114431461759,
  0.016492606762485783,
  0.030193361596525695,
  0.0343811394891945
)
res43_2: Double = 0.378082126688195

The null model of randomly choosing a cover type label and using it as the prediction yields a 37% true positive rate. Makes the 70% TP rate look pretty good. 

## PIPELINES

In [45]:
import org.apache.spark.ml.Pipeline

import org.apache.spark.ml.Pipeline

In [46]:
val inputCols = trainData.columns.filter(_!="Cover_Type")
val assembler = new VectorAssembler().
                    setHandleInvalid("skip").
                    setInputCols(inputCols).
                    setOutputCol("featureVector")

inputCols: Array[String] = Array(
  "Elevation",
  "Aspect",
  "Slope",
  "Horizontal_Distance_To_Hydrology",
  "Vertical_Distance_To_Hydrology",
  "Horizontal_Distance_To_Roadways",
  "Hillshade_9am",
  "Hillshade_Noon",
  "Hillshade_3pm",
  "Horizontal_Distance_To_Fire_Points",
  "Wilderness_Area_0",
  "Wilderness_Area_1",
  "Wilderness_Area_2",
  "Wilderness_Area_3",
  "Soil_Type_0",
  "Soil_Type_1",
  "Soil_Type_2",
  "Soil_Type_3",
  "Soil_Type_4",
  "Soil_Type_5",
  "Soil_Type_6",
  "Soil_Type_7",
  "Soil_Type_8",
  "Soil_Type_9",
  "Soil_Type_10",
  "Soil_Type_11",
  "Soil_Type_12",
  "Soil_Type_13",
  "Soil_Type_14",
  "Soil_Type_15",
  "Soil_Type_16",
  "Soil_Type_17",
  "Soil_Type_18",
  "Soil_Type_19",
  "Soil_Type_20",
  "Soil_Type_21",
  "Soil_Type_22",
  "Soil_Type_23",
...
assembler: VectorAssembler = vecAssembler_cc9f29d6b2dc

In [47]:
val classifer = new DecisionTreeClassifier().
                setSeed(Random.nextLong()).
                setLabelCol("Cover_Type").
                setFeaturesCol("featureVector").
                setPredictionCol("prediction")

classifer: DecisionTreeClassifier = dtc_daa8e5d63cda

The above two transformations, recall that a transformation is a map from one dataframe to another that is part of the DAG, are chained together in a pipeline that is a convenience for defining a series of transforms on the data. We do this to streamline the hyper parameter tuning process.

In [48]:
val pipeline = new Pipeline().setStages(Array(assembler,classifier))

pipeline: Pipeline = pipeline_549c0a5a7904

Use spark.ml.tuning.ParameterBuilder to set up collections of hyper parameters to try. Will obviously need an evaluation metric as well. The ParamGrid BUilder defines the series of values to trye, and the loss function

In [49]:
import org.apache.spark.ml.tuning.{ParamGridBuilder,TrainValidationSplit}

import org.apache.spark.ml.tuning.{ParamGridBuilder,TrainValidationSplit}

In [59]:
val paramGrid = new ParamGridBuilder().
                    addGrid(classifer.impurity, Seq("gini","entropy")).
                    addGrid(classifier.maxDepth, Seq(1,20)).
                    addGrid(classifier.maxBins,Seq(40,300)).
                    addGrid(classifier.minInfoGain, Seq(0.0,0.05)).
                    build()



paramGrid: Array[org.apache.spark.ml.param.ParamMap] = Array(
  {
	dtc_daa8e5d63cda-impurity: gini,
	dtc_47153b85c161-maxBins: 40,
	dtc_47153b85c161-maxDepth: 1,
	dtc_47153b85c161-minInfoGain: 0.0
},
  {
	dtc_daa8e5d63cda-impurity: gini,
	dtc_47153b85c161-maxBins: 40,
	dtc_47153b85c161-maxDepth: 20,
	dtc_47153b85c161-minInfoGain: 0.0
},
  {
	dtc_daa8e5d63cda-impurity: gini,
	dtc_47153b85c161-maxBins: 300,
	dtc_47153b85c161-maxDepth: 1,
	dtc_47153b85c161-minInfoGain: 0.0
},
  {
	dtc_daa8e5d63cda-impurity: gini,
	dtc_47153b85c161-maxBins: 300,
	dtc_47153b85c161-maxDepth: 20,
	dtc_47153b85c161-minInfoGain: 0.0
},
  {
	dtc_daa8e5d63cda-impurity: gini,
	dtc_47153b85c161-maxBins: 40,
	dtc_47153b85c161-maxDepth: 1,
	dtc_47153b85c161-minInfoGain: 0.05
},
  {
	dtc_daa8e5d63cda-impurity: gini,
	dtc_47153b85c161-maxBins: 40,
	dtc_47153b85c161-maxDepth: 20,
	dtc_47153b85c161-minInfoGain: 0.05
},
  {
	dtc_daa8e5d63cda-impurity: gini,
...

In [51]:
val multiclassEval = new MulticlassClassificationEvaluator().
setLabelCol("Cover_Type").
setPredictionCol("prediction").
setMetricName("accuracy")

multiclassEval: MulticlassClassificationEvaluator = mcEval_fd1f6eb4f4b2

In [52]:
val validator = new TrainValidationSplit().
    setSeed(Random.nextLong()).
    setEstimator(pipeline).
    setEvaluator(multiclassEval).
    setEstimatorParamMaps(paramGrid).
    setTrainRatio(0.9)

validator: TrainValidationSplit = tvs_24ab5388caef

Now go ahead and fit the model on all the parameters set in the setEstimatorParamMaps

In [53]:
val validatorModel=validator.fit(trainData)

take at Classifier.scala:111

8 / 8

take at Classifier.scala:111

1 / 1

take at DecisionTreeMetadata.scala:112

...

mapPartitions at RandomForest.scala:538

8 / 8

collectAsMap at RandomForest.scala:557

8 / 8

validatorModel: org.apache.spark.ml.tuning.TrainValidationSplitModel = tvs_24ab5388caef

The validatorModel now contains the best model parameters for the pipeline supplied (in the setEstimator - recall that estimator is one of the spark.ml 'primatives' along with transform and is used for fitting parameters). Because a pipeline was supplied to the estimator the Model returned is wrapped in a pipeline object so we need to get the model out of the pipeline.

In [54]:
import org.apache.spark.ml.PipelineModel

import org.apache.spark.ml.PipelineModel

In [55]:
validatorModel.bestModel.asInstanceOf[PipelineModel].stages.last.extractParamMap

res54: org.apache.spark.ml.param.ParamMap = {
	dtc_47153b85c161-cacheNodeIds: false,
	dtc_47153b85c161-checkpointInterval: 10,
	dtc_47153b85c161-featuresCol: featureVector,
	dtc_47153b85c161-impurity: gini,
	dtc_47153b85c161-labelCol: Cover_Type,
	dtc_47153b85c161-maxBins: 300,
	dtc_47153b85c161-maxDepth: 20,
	dtc_47153b85c161-maxMemoryInMB: 256,
	dtc_47153b85c161-minInfoGain: 0.0,
	dtc_47153b85c161-minInstancesPerNode: 1,
	dtc_47153b85c161-predictionCol: prediction,
	dtc_47153b85c161-probabilityCol: probability,
	dtc_47153b85c161-rawPredictionCol: rawPrediction,
	dtc_47153b85c161-seed: -8289828427721790645
}

Its interesting that the results obtained are very different to the book. This model uses 300 bins whereas the book used only the 40. Gini index was the one that performed the best compared to the entropy that the book found. 0.0 infomration gain threshold was the same (and suggests over fitting).

To get the model out of the pipeline use the code below. note that the code val best= validatorModel.bestModel throws an error. Not clear on why but the code below returns as class with the suffix 'Model' which in spark speak is the trianed model. This should then be useable for making predictions on the test data.

In [56]:
val best = validatorModel.bestModel.asInstanceOf[PipelineModel].stages.last

best: org.apache.spark.ml.Transformer = DecisionTreeClassificationModel (uid=dtc_47153b85c161) of depth 20 with 27491 nodes

In [57]:
val paramsAndMetrics = validatorModel.validationMetrics.zip(validatorModel.getEstimatorParamMaps).sortBy(-_._1)

paramsAndMetrics: Array[(Double, org.apache.spark.ml.param.ParamMap)] = Array(
  (
    0.9068659220006862,
    {
	dtc_daa8e5d63cda-impurity: gini,
	dtc_47153b85c161-maxBins: 300,
	dtc_47153b85c161-maxDepth: 20,
	dtc_47153b85c161-minInfoGain: 0.0
}
  ),
  (
    0.9068659220006862,
    {
	dtc_daa8e5d63cda-impurity: entropy,
	dtc_47153b85c161-maxBins: 300,
	dtc_47153b85c161-maxDepth: 20,
	dtc_47153b85c161-minInfoGain: 0.0
}
  ),
  (
    0.9056841142160039,
    {
	dtc_daa8e5d63cda-impurity: gini,
	dtc_47153b85c161-maxBins: 40,
	dtc_47153b85c161-maxDepth: 20,
	dtc_47153b85c161-minInfoGain: 0.0
}
  ),
  (
    0.9056841142160039,
    {
	dtc_daa8e5d63cda-impurity: entropy,
	dtc_47153b85c161-maxBins: 40,
	dtc_47153b85c161-maxDepth: 20,
	dtc_47153b85c161-minInfoGain: 0.0
}
  ),
  (
    0.6705996721436468,
...

The above .validationMetrics method shows that the top performing parameters result in increbilble close accuarcy scores so the results here are not so different to those in the book.The lesser work 4 bins might end up being a best choice. 

In [58]:
multiclassEval.evaluate(best.transform(testData))

: 

In [60]:
val assembledTestData = assembler.setHandleInvalid("skip").transform(testData)

assembledTestData: DataFrame = [Elevation: int, Aspect: int ... 54 more fields]

Note: because I extracted a model from the pipelineModel returned by the trainvalidationsplit training on the parameter grid then I need to assemble the data into a vector. If it was a pipelineModel ( a full pipeline returend from the training) then this would have been handled by the assembler stage in the pipeline I build.

In [61]:
multiclassEval.evaluate(best.transform(assembledTestData))

map at MulticlassMetrics.scala:45

8 / 8

collectAsMap at MulticlassMetrics.scala:47

8 / 8

countByValue at MulticlassMetrics.scala:42

8 / 8

countByValue at MulticlassMetrics.scala:42

8 / 8

res60: Double = 0.910148975791434

The accuracy went up to 90%!

In [62]:
multiclassEval.evaluate(best.transform(assembledTrainData))

map at MulticlassMetrics.scala:45

8 / 8

collectAsMap at MulticlassMetrics.scala:47

8 / 8

countByValue at MulticlassMetrics.scala:42

8 / 8

countByValue at MulticlassMetrics.scala:42

8 / 8

res61: Double = 0.9438430181868241

The above analysis used one hot encoding of the categorical vectors. One hot encoding treats each possible value of the categorical variable as an independant feature that can take values between 0 and 1 even though these are the only two values possilble. The description in the book suggests that there is information inefficiency in doing this and that registering them as categorical variables does improve accuracy. One explaination might be the reduction in the dimension of the problem if one does not one-hot encode things. The process of registering as a categorical variable is to replace the one hot encoded data with a column whose value is a vector of length equal to the number of categories PLUS using VectorIndexer to register metadata (see pages 87 and 88 of the book for details of implementing this)

### Random Forest

In [63]:
import org.apache.spark.ml.classification.RandomForestClassifier

import org.apache.spark.ml.classification.RandomForestClassifier

In [64]:
val classifierRF = new org.apache.spark.ml.classification.RandomForestClassifier().
                setSeed(Random.nextLong()).
                setLabelCol("Cover_Type").
                setFeaturesCol("featureVector").
                setPredictionCol("prediction")

classifierRF: RandomForestClassifier = rfc_7952718418f9

In [65]:
val fittedRF = classifierRF.fit(assembledTrainData)

take at Classifier.scala:111

8 / 8

take at Classifier.scala:111

1 / 1

take at DecisionTreeMetadata.scala:112

...

collectAsMap at RandomForest.scala:557

8 / 8

first at RandomForestClassifier.scala:145

1 / 1

fittedRF: org.apache.spark.ml.classification.RandomForestClassificationModel = RandomForestClassificationModel (uid=rfc_7952718418f9) with 20 trees

In [66]:
val predsRF = fittedRF.transform(assembledTrainData)

predsRF: DataFrame = [Elevation: int, Aspect: int ... 57 more fields]

In [67]:
predsRF.select("Cover_Type","prediction","probability").show(truncate=false)

show at cmd66.sc:1

1 / 1

+----------+----------+---------------------------------------------------------------------------------------------------------------------------------------------------+
|Cover_Type|prediction|probability                                                                                                                                        |
+----------+----------+---------------------------------------------------------------------------------------------------------------------------------------------------+
|6.0       |3.0       |[0.0,0.018334941021530295,0.12401552712518686,0.5352142664042259,0.05786744740653234,0.006653935310713508,0.25725119164453825,6.626910872727582E-4]|
|6.0       |3.0       |[0.0,0.05283504694425428,0.20637876356047788,0.38305100384433344,0.13982672574501787,0.006070888277339529,0.20958373440912653,0.002253837219450328]|
|6.0       |3.0       |[0.0,0.052835046944254296,0.20637638931677893,0.4425958700152107,0.044582180403894625,0.00607088827733953,0.245285787

In [68]:
predsRF.select("Cover_Type","prediction").groupBy("Cover_Type").pivot("prediction",(1 to 7)).count().na.fill(0).sort("cover_type").show()

show at cmd67.sc:1

8 / 8

show at cmd67.sc:1

200 / 200

show at cmd67.sc:1

200 / 200

+----------+------+------+-----+---+---+---+---+
|Cover_Type|     1|     2|    3|  4|  5|  6|  7|
+----------+------+------+-----+---+---+---+---+
|       1.0|140772| 49993|    0|  0|  0|  0|  0|
|       2.0| 59143|193516| 2082|  0|  0|  0|  0|
|       3.0|     0| 10873|21476|  1|  0|  0|  0|
|       4.0|     0|    12| 2450| 32|  0|  0|  0|
|       5.0|     0|  8536|    0|  0|  0|  0|  0|
|       6.0|     0|  6898| 8717|  0|  0|  0|  0|
|       7.0| 18434|    81|    0|  0|  0|  0|  0|
+----------+------+------+-----+---+---+---+---+



Note: Without setting the defaults for impurity,infogain etc the confusion matrix shows no predictions for 4,5,6,7. The decisiontreeclassifier showed few predictions in this area but certainly more than zero. With them set to the values found for the descisiontree classifier then the results look much better

In [69]:
val evalRF = new org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator().
                        setLabelCol("Cover_Type").setPredictionCol("prediction").setMetricName("accuracy")


evalRF: MulticlassClassificationEvaluator = mcEval_7c1dcc17bb4f

In [70]:
evalRF.evaluate(predsRF)

map at MulticlassMetrics.scala:45

8 / 8

collectAsMap at MulticlassMetrics.scala:47

8 / 8

countByValue at MulticlassMetrics.scala:42

8 / 8

countByValue at MulticlassMetrics.scala:42

8 / 8

res69: Double = 0.6802774676109336

Note: Out of the box the random forest classifier resulted in an accuracy of 67%!!! 84% was acheived with the tuning due to setting the deafult hyperparameters to that of the decision tree optimal values.

In [71]:
fittedRF.explainParams

res70: String = """cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. (default: false)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n]. (default: auto)
featuresCol: features column name (default: features, current: featureVector)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini (default: gini)
labelCol: label column name (default: label, current: Cover_Type)
maxBins: Max number of bins for discretizing continuo

Try some hyper parameter tuning. Create the pipeline; create the ParamGridBuilder; addGrid to set parameters of interest; create evaluator; use a TrainValidationSplit instance to run the modell fititng on the grid. This will return a list of fitted Pipelines with the best pipeline being accessible from the .stages.last. The model then needs to be extracted.

In [72]:
val rfPipeline = new org.apache.spark.ml.Pipeline()
rfPipeline.setStages(Array(assembler,classifierRF))


rfPipeline: Pipeline = pipeline_8b74a3aa1b2b
res71_1: Pipeline = pipeline_8b74a3aa1b2b

Try changing the number of trees only. 

In [73]:
val paramGridRF = new ParamGridBuilder().
                addGrid(classifierRF.numTrees, Seq(20,30)).
                build()

paramGridRF: Array[org.apache.spark.ml.param.ParamMap] = Array(
  {
	rfc_7952718418f9-numTrees: 20
},
  {
	rfc_7952718418f9-numTrees: 30
}
)

In [74]:
evalRF.explainParams

res73: String = """labelCol: label column name (default: label, current: Cover_Type)
metricName: metric name in evaluation (f1|weightedPrecision|weightedRecall|accuracy) (default: f1, current: accuracy)
predictionCol: prediction column name (default: prediction, current: prediction)"""

In [75]:
val validatorRF = new TrainValidationSplit().
                        setSeed(Random.nextLong()).
                        setEstimator(rfPipeline).
                        setEvaluator(evalRF).setEstimatorParamMaps(paramGridRF).
                        setTrainRatio(0.9)

validatorRF: TrainValidationSplit = tvs_a9d99b98b4c1

In [76]:
val validatorModel = validatorRF.fit(trainData)

take at Classifier.scala:111

8 / 8

take at Classifier.scala:111

1 / 1

take at DecisionTreeMetadata.scala:112

...

collectAsMap at RandomForest.scala:557

8 / 8

first at RandomForestClassifier.scala:145

1 / 1

validatorModel: org.apache.spark.ml.tuning.TrainValidationSplitModel = tvs_a9d99b98b4c1

In [77]:
validatorModel.validationMetrics.zip(validatorModel.getEstimatorParamMaps).sortBy(-_._1)

res76: Array[(Double, org.apache.spark.ml.param.ParamMap)] = Array(
  (
    0.6786903440621531,
    {
	rfc_7952718418f9-numTrees: 20
}
  ),
  (
    0.678498985801217,
    {
	rfc_7952718418f9-numTrees: 30
}
  )
)

The above tuning doesn't improve the random forers performance even if replicating the decision tree. 

In [78]:
validatorModel.bestModel.asInstanceOf[PipelineModel].stages.last.extractParamMap

res77: org.apache.spark.ml.param.ParamMap = {
	rfc_7952718418f9-cacheNodeIds: false,
	rfc_7952718418f9-checkpointInterval: 10,
	rfc_7952718418f9-featureSubsetStrategy: auto,
	rfc_7952718418f9-featuresCol: featureVector,
	rfc_7952718418f9-impurity: gini,
	rfc_7952718418f9-labelCol: Cover_Type,
	rfc_7952718418f9-maxBins: 32,
	rfc_7952718418f9-maxDepth: 5,
	rfc_7952718418f9-maxMemoryInMB: 256,
	rfc_7952718418f9-minInfoGain: 0.0,
	rfc_7952718418f9-minInstancesPerNode: 1,
	rfc_7952718418f9-numTrees: 20,
	rfc_7952718418f9-predictionCol: prediction,
	rfc_7952718418f9-probabilityCol: probability,
	rfc_7952718418f9-rawPredictionCol: rawPrediction,
	rfc_7952718418f9-seed: -1447657657160873267,
	rfc_7952718418f9-subsamplingRate: 1.0
}

Can't get the random forest to outperfrom the decision tree! Stackoverflow suggests that if the subtrees are stable then the rf and the decision trees perform the same. 


Might be the case that one hot encoding is reducing the performance. Will try the same experiments with usnig indexedVector representation of the categorical variables. 

## Recode categoricals instead of using onehot encoding - Decision tree example

In [79]:
import org.apache.spark.sql.functions._
import org.apache.spark.ml.linalg.Vector // this required so that spark.ml.Vector type is imported. Fall back si the scala vector which takes a type

import org.apache.spark.sql.functions._

import org.apache.spark.ml.linalg.Vector // this required so that spark.ml.Vector type is imported. Fall back si the scala vector which takes a type

The code below initially failed because the vector class I was importing was spark.mllib.linalg.Vector not spark.ml.linalg.Vector

The purpose of this method is to overwrite the n one-hot encoded columns with the a single vector column to represent the one hot encoded values in a single field. Vector Indexer, apparently, attaches certain meta-data that avoids this column being treated as a numeric column as in the case of one hot encoding.


In [80]:
def unencodeOneHot(soilColCount: Int)(data: DataFrame):DataFrame ={
    val wildernessCols=(0 until 4).map(i=>s"Wilderness_Area_$i").toArray
    
    //important that invalid inputs be handled by invoking setHandleInvalid.Otherwise downstream errors throw error
    val wildernessAssembler = new VectorAssembler().
                    setInputCols(wildernessCols).setHandleInvalid("skip").
                    setOutputCol("wilderness")
    
    // This UDF used to convert multiple columns of one-hot encoded data to a single vector column
    val unhotUDF = udf((vec: Vector) => vec.toArray.indexOf(1.0).toDouble)
    
    val withWilderness =   wildernessAssembler.transform(data).
                            drop(wildernessCols:_*).
                            withColumn("wilderness",unhotUDF($"wilderness"))
    

   val soilCols = (0 until soilColCount).map(i => s"Soil_Type_$i").toArray
    
   val soilAssembler = new VectorAssembler().setHandleInvalid("skip").
            setInputCols(soilCols).
            setOutputCol("soil")
    
   return soilAssembler.transform(withWilderness).
            drop(soilCols:_*).
            withColumn("soil", unhotUDF($"soil")) 
}

defined function unencodeOneHot

In [81]:
val unencodedTrainData = unencodeOneHot(40)(trainData)
//unencodedTrainData.columns.foreach(println)
unencodedTrainData.count()
trainData.count()

count at cmd80.sc:3

8 / 8

count at cmd80.sc:3

1 / 1

count at cmd80.sc:4

8 / 8

count at cmd80.sc:4

1 / 1

unencodedTrainData: DataFrame = [Elevation: int, Aspect: int ... 11 more fields]
res80_1: Long = 523016L
res80_2: Long = 523233L

Test objects for testing udf


org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)
case class onehot(Wilderness_Area_0:Double,Wilderness_Area_1:Double,Wilderness_Area_2:Double,Wilderness_Area_3:Double, Soil_Type_0:Double,Soil_Type_1:Double)


val test = Seq(onehot(0,1,0,0,1,0),onehot(0,0,1,0,0,1),onehot(1,0,0,0,1,0),onehot(0,0,0,1,0,1)).toDF()



test.show()

test.printSchema

unencodeOneHot(2)(test).show()

Recreate the data with the indexed vector column repalcing the one hot encoded. Indexer (defined below) is a transformation and is added to the pipeline to convert the existing data to a form the classifier can use.

In [82]:
import org.apache.spark.ml.feature.VectorIndexer

import org.apache.spark.ml.feature.VectorIndexer

In [83]:
//Indexer transformation to replace the columsn wiht a single column of type vector that captures the categorical values
val inputCols = unencodedTrainData.columns.filter(_ != "Cover_Type")

val rfCat_Assembler = new VectorAssembler().
                    setHandleInvalid("skip").
                    setInputCols(inputCols).
                    setOutputCol("featureVector")

val rfCat_Indexer = new VectorIndexer().
                setMaxCategories(40).
                setInputCol("featureVector").
                setOutputCol("indexedVector")

//change the classifier configuration to take the indexedVector (categorical values non-onehot encoded)
val rfCat_Classifier = new DecisionTreeClassifier().
                    setSeed(Random.nextLong()).
                    setLabelCol("Cover_Type").
                    setFeaturesCol("featureVector").
                    setPredictionCol("prediction").
                setImpurity("gini").setMaxBins(300).setMaxDepth(20).setMinInfoGain(0.0)

//val pipeline_cat = new Pipeline().setStages(Array(assembler, classifierRF_cat))
val rfCat_Pipeline = new Pipeline().setStages(Array(rfCat_Assembler, rfCat_Indexer, rfCat_Classifier))

inputCols: Array[String] = Array(
  "Elevation",
  "Aspect",
  "Slope",
  "Horizontal_Distance_To_Hydrology",
  "Vertical_Distance_To_Hydrology",
  "Horizontal_Distance_To_Roadways",
  "Hillshade_9am",
  "Hillshade_Noon",
  "Hillshade_3pm",
  "Horizontal_Distance_To_Fire_Points",
  "wilderness",
  "soil"
)
rfCat_Assembler: VectorAssembler = vecAssembler_6cce899514f6
rfCat_Indexer: VectorIndexer = vecIdx_3db47557759f
rfCat_Classifier: DecisionTreeClassifier = dtc_ab5d9eb8a2a7
rfCat_Pipeline: Pipeline = pipeline_4f018dc231eb

We now have a pipeline that we can use to run the parameter tuning.

In [84]:
val validatorRF_Cat = new TrainValidationSplit().
                        setSeed(Random.nextLong()).
                        setEstimator(rfCat_Pipeline).
                        setEvaluator(evalRF).setEstimatorParamMaps(paramGridRF).
                        setTrainRatio(0.9)

validatorRF_Cat: TrainValidationSplit = tvs_e6171a3ba401

In [85]:
val validatorModelRF_cat = validatorRF_Cat.fit(unencodedTrainData)

take at VectorIndexer.scala:145

1 / 1

reduce at VectorIndexer.scala:154

8 / 8

take at Classifier.scala:111

...

mapPartitions at RandomForest.scala:538

8 / 8

collectAsMap at RandomForest.scala:557

8 / 8

validatorModelRF_cat: org.apache.spark.ml.tuning.TrainValidationSplitModel = tvs_e6171a3ba401

Get the model valuation metrics out of it

In [86]:
validatorModelRF_cat.validationMetrics.zip(validatorModelRF_cat.getEstimatorParamMaps).sortBy(-_._1)

res85: Array[(Double, org.apache.spark.ml.param.ParamMap)] = Array(
  (
    0.9183031001055764,
    {
	rfc_7952718418f9-numTrees: 20
}
  ),
  (
    0.9183031001055764,
    {
	rfc_7952718418f9-numTrees: 30
}
  )
)

In [87]:
validatorModelRF_cat.bestModel.asInstanceOf[PipelineModel].stages.last.extractParamMap

res86: org.apache.spark.ml.param.ParamMap = {
	dtc_ab5d9eb8a2a7-cacheNodeIds: false,
	dtc_ab5d9eb8a2a7-checkpointInterval: 10,
	dtc_ab5d9eb8a2a7-featuresCol: featureVector,
	dtc_ab5d9eb8a2a7-impurity: gini,
	dtc_ab5d9eb8a2a7-labelCol: Cover_Type,
	dtc_ab5d9eb8a2a7-maxBins: 300,
	dtc_ab5d9eb8a2a7-maxDepth: 20,
	dtc_ab5d9eb8a2a7-maxMemoryInMB: 256,
	dtc_ab5d9eb8a2a7-minInfoGain: 0.0,
	dtc_ab5d9eb8a2a7-minInstancesPerNode: 1,
	dtc_ab5d9eb8a2a7-predictionCol: prediction,
	dtc_ab5d9eb8a2a7-probabilityCol: probability,
	dtc_ab5d9eb8a2a7-rawPredictionCol: rawPrediction,
	dtc_ab5d9eb8a2a7-seed: -3616907792373549603
}